In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [ ]:
import re

In [ ]:
path_to_file = tf.keras.utils.get_file('harrypotter.txt','https://raw.githubusercontent.com/GokuMohandas/madewithml/main/datasets/harrypotter.txt')

1146880/1139144 [==============================] - 0s 0us/step


In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='cp1252')

In [ ]:
len(text)

1139144

In [ ]:
text[:250]

"Harry Potter and the Deathly Hallows By J. K. Rowling\r\n\r\nChapter One The Dark Lord Ascending\r\nThe two men appeared out of nowhere, a few yards apart in the narrow, moonlit lane. For a second they stood quite still, wands directed at each other's ches"

In [ ]:
a=text.split(' ')

In [ ]:
for i in a:
  print(a)
  break

['Harry', 'Potter', 'and', 'the', 'Deathly', 'Hallows', 'By', 'J.', 'K.', 'Rowling\r\n\r\nChapter', 'One', 'The', 'Dark', 'Lord', 'Ascending\r\nThe', 'two', 'men', 'appeared', 'out', 'of', 'nowhere,', 'a', 'few', 'yards', 'apart', 'in', 'the', 'narrow,', 'moonlit', 'lane.', 'For', 'a', 'second', 'they', 'stood', 'quite', 'still,', 'wands', 'directed', 'at', 'each', "other's", 'chests;', 'then,', 'recognizing', 'each', 'other,', 'they', 'stowed', 'their', 'wands', 'beneath', 'their', 'cloaks', 'and', 'started', 'walking', 'briskly', 'in', 'the', 'same', 'direction.', '"News?"', 'asked', 'the', 'taller', 'of', 'the', 'two.', '"The', 'best,"', 'replied', 'Severus', 'Snape.', 'The', 'lane', 'was', 'bordered', 'on', 'the', 'left', 'by', 'wild,', 'low-growing', 'brambles,', 'on', 'the', 'right', 'by', 'a', 'high,', 'neatly', 'manicured', 'hedge.', 'The', "men's", 'long', 'cloaks', 'flapped', 'around', 'their', 'ankles', 'as', 'they', 'marched.', '"Thought', 'I', 'might', 'be', 'late,"', 'sai

In [ ]:
def preprocess(w):
  w=re.sub(r"([?.!,%&^%$#@])",r" ",w)
#  w=re.sub(r'[" "]+'," ",w)
  w=re.sub(r"[^a-zA-Z?.!,]+"," ",w)
  #w=w.strip()
  w=w.replace('\r','')
  w=w.replace('\n','')
  w=w.lower()
  return w


In [ ]:
ddata=list(map(lambda x:preprocess(x),a))

In [ ]:
c=' '
c=c.join(ddata)

In [ ]:
harry_text=c

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer=Tokenizer(char_level=True)
tokenizer.fit_on_texts(harry_text)
sequences=tokenizer.texts_to_sequences(harry_text)
word_index=tokenizer.word_index

In [ ]:
harry_text

'harry potter and the deathly hallows by j  k  rowling chapter one the dark lord ascending the two men appeared out of nowhere  a few yards apart in the narrow  moonlit lane  for a second they stood quite still  wands directed at each other s chests  then  recognizing each other  they stowed their wands beneath their cloaks and started walking briskly in the same direction   news  asked the taller of the two   the best  replied severus snape  the lane was bordered on the left by wild  low growing brambles  on the right by a high  neatly manicured hedge  the men s long cloaks flapped around their ankles as they marched   thought i might be late  said yaxley  his blunt features sliding in and out of sight as the branches of overhanging trees broke the moonlight   it was a little trickier than i expected  but i hope he will be satisfied  you sound confident that your reception will be good  snape nodded  but did not elaborate  they turned right  into a wide driveway that led off the lane 

In [ ]:
sequences

In [ ]:
len(sequences)

1113984

In [ ]:
word_index

In [ ]:
seq_length=100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
examples_per_epoch

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(sequences)

In [ ]:
for i in char_dataset.take(5):
    print(i.numpy())

[6]
[4]
[7]
[7]
[17]


In [ ]:
sequences1 = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
list1=[1,2,3]
list1[:-1]

[1, 2]

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences1.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print(len(target_example.numpy()))

100


In [ ]:
next(iter(dataset))

In [ ]:
BATCH_SIZE=64
BUFFER_SIZE=10000
dataset1 = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
next(iter(dataset1))

In [ ]:
vocab=len(word_index)


In [ ]:
vocab

27

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(28,256,input_shape=[None,]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True)),
    tf.keras.layers.LSTM(512,return_sequences=True,recurrent_initializer='glorot_uniform',recurrent_dropout=0.3,recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01)),
  #  tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(28),
   # tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab+1,activation='softmax')),
    ])

In [ ]:

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, None, 256)         7168      
_________________________________________________________________
bidirectional_18 (Bidirectio (None, None, 2048)        10493952  
_________________________________________________________________
lstm_39 (LSTM)               (None, None, 512)         5244928   
_________________________________________________________________
dense_54 (Dense)             (None, None, 256)         131328    
_________________________________________________________________
dense_55 (Dense)             (None, None, 64)          16448     
_________________________________________________________________
dense_56 (Dense)             (None, None, 28)          1820      
Total params: 15,895,644
Trainable params: 15,895,644
Non-trainable params: 0
_________________________________________

In [ ]:
import os

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size+1)
    ])
    return model

In [ ]:
model1= build_model(
    vocab_size=vocab,
    embedding_dim=256,
    rnn_units=1024,
    batch_size=BATCH_SIZE)

In [ ]:
model1.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model1.fit(dataset1, epochs=3, callbacks=[checkpoint_callback])

Epoch 1/3
172/172 [==============================] - 11s 52ms/step - loss: 2.8896 - accuracy: 0.2647
Epoch 2/3
172/172 [==============================] - 10s 53ms/step - loss: 1.8854 - accuracy: 0.4300
Epoch 3/3
172/172 [==============================] - 10s 53ms/step - loss: 1.5487 - accuracy: 0.5291


In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [ ]:
idx2char={}
for i,j in word_index.items():
  idx2char[j]=i


In [ ]:
for input_example_batch, target_example_batch in dataset1.take(1):
    example_batch_predictions = model(input_example_batch)
    print(input_example_batch, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[[[ 1]
  [ 5]
  [13]
  ...
  [ 4]
  [ 8]
  [ 1]]

 [[14]
  [ 4]
  [10]
  ...
  [11]
  [ 9]
  [ 2]]

 [[ 4]
  [15]
  [ 4]
  ...
  [ 1]
  [19]
  [ 2]]

 ...

 [[ 2]
  [11]
  [ 1]
  ...
  [14]
  [ 1]
  [11]]

 [[ 2]
  [ 1]
  [ 4]
  ...
  [ 2]
  [16]
  [ 5]]

 [[ 9]
  [15]
  [ 6]
  ...
  [ 6]
  [ 4]
  [ 7]]], shape=(64, 100, 1), dtype=int32) # (batch_size, sequence_length, vocab_size)


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
nn=list(sampled_indices)

In [ ]:
nn

[5,
 13,
 3,
 1,
 4,
 1,
 6,
 5,
 14,
 12,
 1,
 5,
 19,
 1,
 10,
 6,
 5,
 18,
 22,
 1,
 4,
 8,
 11,
 1,
 16,
 9,
 10,
 2,
 7,
 17,
 1,
 1,
 4,
 8,
 11,
 1,
 3,
 6,
 7,
 2,
 14,
 1,
 6,
 9,
 16,
 10,
 2,
 12,
 19,
 1,
 21,
 4,
 18,
 22,
 1,
 5,
 8,
 3,
 5,
 1,
 3,
 6,
 2,
 1,
 15,
 7,
 5,
 13,
 8,
 11,
 1,
 9,
 3,
 1,
 3,
 5,
 5,
 22,
 1,
 3,
 6,
 2,
 16,
 1,
 8,
 2,
 4,
 7,
 12,
 17,
 1,
 6,
 4,
 12,
 19,
 1,
 4,
 8,
 1,
 10]

In [ ]:
np.array(sampled_indices)

array([ 5, 13,  3,  1,  4,  1,  6,  5, 14, 12,  1,  5, 19,  1, 10,  6,  5,
       18, 22,  1,  4,  8, 11,  1, 16,  9, 10,  2,  7, 17,  1,  1,  4,  8,
       11,  1,  3,  6,  7,  2, 14,  1,  6,  9, 16, 10,  2, 12, 19,  1, 21,
        4, 18, 22,  1,  5,  8,  3,  5,  1,  3,  6,  2,  1, 15,  7,  5, 13,
        8, 11,  1,  9,  3,  1,  3,  5,  5, 22,  1,  3,  6,  2, 16,  1,  8,
        2,  4,  7, 12, 17,  1,  6,  4, 12, 19,  1,  4,  8,  1, 10])

In [ ]:
idx2char[nn[0]]

'o'

In [ ]:
nn

In [ ]:
a=[]
for i in nn:
  a.append(idx2char[i])
  


In [ ]:
a

['o',
 'u',
 't',
 ' ',
 'a',
 ' ',
 'h',
 'o',
 'w',
 'l',
 ' ',
 'o',
 'f',
 ' ',
 's',
 'h',
 'o',
 'c',
 'k',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 'm',
 'i',
 's',
 'e',
 'r',
 'y',
 ' ',
 ' ',
 'a',
 'n',
 'd',
 ' ',
 't',
 'h',
 'r',
 'e',
 'w',
 ' ',
 'h',
 'i',
 'm',
 's',
 'e',
 'l',
 'f',
 ' ',
 'b',
 'a',
 'c',
 'k',
 ' ',
 'o',
 'n',
 't',
 'o',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'g',
 'r',
 'o',
 'u',
 'n',
 'd',
 ' ',
 'i',
 't',
 ' ',
 't',
 'o',
 'o',
 'k',
 ' ',
 't',
 'h',
 'e',
 'm',
 ' ',
 'n',
 'e',
 'a',
 'r',
 'l',
 'y',
 ' ',
 'h',
 'a',
 'l',
 'f',
 ' ',
 'a',
 'n',
 ' ',
 's']

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [word_index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        print(idx2char[predicted_id])
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"harry potter"))